In [1]:
import os

# Check if the current directory is 'notebooks'
if os.path.basename(os.getcwd()) == 'notebooks':
    # Change to the parent directory
    os.chdir('..')

In [2]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
# import kalshi_python

# starter client: recommended for all levels of programming experience (what this client is implemented using)
import time
import uuid

from kalshi_tracker.kalshi import get_kalshi

In [3]:
exchange_client = get_kalshi()

In [4]:
# first we will check on the exchange status to confirm you are properly connected...
print(exchange_client.get_exchange_status())

{'exchange_active': True, 'trading_active': True}


In [5]:
# You can discover markets through the get_markets endpoint...

# and use query parameters to filter your search!
market_params = {
    "limit": 100,
    "cursor": None,  # passing in the cursor from the previous get_markets call
    "event_ticker": None,
    "series_ticker": None,
    "max_close_ts": None,  # pass in unix_ts
    "min_close_ts": None,  # pass in unix_ts
    "status": None,
    "tickers": None,
}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response["cursor"]

print("keys:", markets_response.keys())
print()
print("number of objects:", len(markets_response["markets"]))  # 100 objects!
print()
print("first market in payload:", markets_response["markets"][0])
print()
print("cursor:", cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in payload: {'ticker': 'KXSECHUD-26DEC31-ST', 'event_ticker': 'KXSECHUD-26DEC31', 'market_type': 'binary', 'title': 'Will Scott Turner be confirmed as Secretary of Housing and Urban Development by Dec 31, 2026?', 'subtitle': 'Scott Turner', 'yes_sub_title': 'Scott Turner', 'no_sub_title': 'Scott Turner', 'open_time': '2024-11-23T14:35:26Z', 'close_time': '2027-01-01T04:59:00Z', 'expected_expiration_time': '2027-01-01T15:00:00Z', 'expiration_time': '2027-01-07T15:00:00Z', 'latest_expiration_time': '2027-01-07T15:00:00Z', 'settlement_timer_seconds': 3600, 'status': 'active', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 2, 'yes_ask': 98, 'no_bid': 2, 'no_ask': 98, 'last_price': 99, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 5, 'volume_24h': 5, 'liquidity': 565532, 'open_interest': 5, 'result': '', 'can_close_early': True, 'expiration_value

In [6]:
# What are cursors and how do they work?

# The Cursor represents a pointer to the next page of records in the pagination.
# So this optional parameter, when filled, should be filled with the cursor string returned in a previous request to this end-point.
# Filling this would basically tell the api to get the next page containing the number of records passed on the limit parameter.
# On the other side not filling it tells the api you want to get the first page for another query.
# The cursor does not store any filters, so if any filter parameters like tickers, max_ts or min_ts were passed in the original query they must be passed again.

# Let's try it in action! Suppose we wanted to get the next 100 market objects...

market_params = {
    "limit": 100,
    "cursor": cursor,  # passing in the cursor from the previous get_markets call
    "event_ticker": None,
    "series_ticker": None,
    "max_close_ts": None,  # pass in unix_ts
    "min_close_ts": None,  # pass in unix_ts
    "status": None,
    "tickers": None,
}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response["cursor"]

print("keys:", markets_response.keys())
print()
print("number of objects:", len(markets_response["markets"]))  # 100 objects!
print()
print(
    "first market in market_response payload:",
    markets_response["markets"][0],
)  # new markets!
print()
print("new cursor!", cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in market_response payload: {'ticker': 'KXSPOTIFYD-24NOV22-R', 'event_ticker': 'KXSPOTIFYD-24NOV22', 'market_type': 'binary', 'title': 'Will the top song on Nov 22, 2024 be reincarnated?', 'subtitle': ':: Kendrick Lamar', 'yes_sub_title': 'reincarnated', 'no_sub_title': 'reincarnated', 'open_time': '2024-11-22T17:45:00Z', 'close_time': '2024-11-23T04:59:00Z', 'expected_expiration_time': '2024-11-24T15:00:00Z', 'expiration_time': '2024-12-06T15:00:00Z', 'latest_expiration_time': '2024-12-06T15:00:00Z', 'settlement_timer_seconds': 300, 'status': 'finalized', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 1, 'no_bid': 99, 'no_ask': 100, 'last_price': 1, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 1031, 'volume_24h': 1031, 'liquidity': 0, 'open_interest': 1025, 'result': 'no', 'settlement_value': 0, 'can_close_early': True, 'expi

In [7]:
# Next, let's look at event level data by passing an event ticker to the get_event endpoint...

event_ticker = markets_response["markets"][5]["event_ticker"]
event_params = {"event_ticker": event_ticker}
event_response = exchange_client.get_event(**event_params)

print("keys:", event_response.keys())
print()
print("event object:", event_response["event"])
print()
print("first market in event_response payload:", event_response["markets"][0])

keys: dict_keys(['event', 'markets'])

event object: {'event_ticker': 'KXSPOTIFYD-24NOV22', 'series_ticker': 'KXSPOTIFYD', 'sub_title': 'On Nov 22, 2024 Chart', 'title': 'Top song on Spotify USA Chart on Nov 22, 2024?', 'mutually_exclusive': True, 'category': 'Entertainment'}

first market in event_response payload: {'ticker': 'KXSPOTIFYD-24NOV22-THA', 'event_ticker': 'KXSPOTIFYD-24NOV22', 'market_type': 'binary', 'title': '', 'subtitle': ':: Gracie Abrams', 'yes_sub_title': "That's So True", 'no_sub_title': "That's So True", 'open_time': '2024-11-21T16:30:00Z', 'close_time': '2024-11-23T04:59:00Z', 'expected_expiration_time': '2024-11-24T15:00:00Z', 'expiration_time': '2024-12-06T15:00:00Z', 'latest_expiration_time': '2024-12-06T15:00:00Z', 'settlement_timer_seconds': 300, 'status': 'finalized', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 4, 'yes_ask': 14, 'no_bid': 86, 'no_ask': 96, 'last_price': 14, 'previous_yes_bid': 0, 'previous_yes_ask':

In [8]:
# Next, let's look at series level data by passing a series ticker to the get_series endpoint!
series_ticker = event_response["event"]["series_ticker"]
series_params = {"series_ticker": series_ticker}
series_response = exchange_client.get_series(**series_params)

print("keys:", series_response.keys())
print()
print("series object:", series_response["series"])
print()

keys: dict_keys(['series'])

series object: {'ticker': 'KXSPOTIFYD', 'frequency': 'daily', 'title': 'Daily USA Spotify chart', 'category': 'Entertainment', 'tags': ['Music', 'Music charts'], 'settlement_sources': [{'url': 'https://charts.spotify.com/charts/view/regional-us-daily/latest', 'name': 'Spotify'}], 'contract_url': 'https://kalshi-public-docs.s3.us-east-1.amazonaws.com/regulatory/product-certifications/SPOTIFYCHART.pdf'}



In [9]:
# Next let's look at the recent market history for a market
ticker = "NGDP-22-C7.5"

market_history_params = {
    "ticker": ticker,
    "limit": 100,
    "cursor": None,
    "max_ts": None,  # pass in unix_ts
    "min_ts": round(time.time() - 1000000),  # passing a recent unix_ts
}
market_history_response = exchange_client.get_market_history(**market_history_params)

print("keys:", market_history_response.keys())
print()
print("most recent market history object:", market_history_response["history"][-1])
print()

# and then also look at the most current view of the orderbook
market_history_params = {"ticker": ticker, "depth": 30}
orderbook_response = exchange_client.get_orderbook(**market_history_params)

print("keys:", orderbook_response.keys())
print()
print("orderbook object:", orderbook_response)
print()

HttpError: HttpError(404 Not Found)

In [ ]:
# Now let's suppose we wanted to place a trade on one of these markets...
# to do so, we would first want to check out available balance...

current_balance = exchange_client.get_balance()
current_balance

In [ ]:
# Now that you have some balance, you might want to see how your current positions are doing...

positions_params = {
    "limit": None,
    "cursor": None,
    "settlement_status": None,
    "ticker": None,
    "event_ticker": None,
}

current_position = exchange_client.get_positions(**positions_params)
current_position

In [ ]:
# seems like some of your recent orders had been filled. To check on those we use the get_positions endpoint ...

fills_params = {
    "ticker": None,
    "order_id": None,
    "min_ts": None,
    "max_ts": None,
    "limit": None,
    "cursor": None,
}

fills = exchange_client.get_fills(**fills_params)
fills

In [ ]:
# you may even want to check on some of your recent positions settled...

settlement_params = {"limit": None, "cursor": None}

settlements = exchange_client.get_portfolio_settlements(**settlement_params)
settlements

In [ ]:
# Now onto placing an order...
# There are many different ways to think about placing orders at Kalshi.
# The following param examples will walk through some of those

# Limit buy order for 10 units at 30c No on GDPW-22-A3

ticker = "TESTING-5"

order_params = {
    "ticker": ticker,
    "client_order_id": str(uuid.uuid4()),
    "type": "limit",
    "action": "buy",
    "side": "no",
    "count": 10,
    "yes_price": None,  # yes_price = 100 - no_price
    "no_price": 30,  # no_price = 100 - yes_price
    "expiration_ts": None,
    "sell_position_floor": None,
    "buy_max_cost": None,
}

exchange_client.create_order(**order_params)

# EQUIVALENTLY, because buying No is equivalent to selling yes...

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'limit',
#                     'action':'sell',
#                     'side':'yes',
#                     'count':10,
#                     'yes_price':None, # yes_price = 100 - no_price
#                     'no_price':30, # no_price = 100 - yes_price
#                     'expiration_ts':None,
#                     'sell_position_floor':None,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)


# # Market sell order for 12 units Yes on GDPW-22-A3, without flipping position

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'market',
#                     'action':'sell',
#                     'side':'yes',
#                     'count':12,
#                     'yes_price':1,
#                     'no_price':None,
#                     'expiration_ts':None,
#                     'sell_position_floor':0,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)

# # EQUIVALENTLY, because buying No is equivalent to selling yes...

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'market',
#                     'action':'buy',
#                     'side':'no',
#                     'count':12,
#                     'yes_price':1,
#                     'no_price':None,
#                     'expiration_ts':None,
#                     'sell_position_floor':0,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)
